In [135]:
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, XSD

# Define the namespaces
ear = Namespace("http://www.essepuntato.it/2008/12/earmark#")
whata = Namespace("https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#")
per= Namespace('http://www.ontologydesignpatterns.org/ont/persp/perspectivisation.owl#')
dns= Namespace('http://www.ontologydesignpatterns.org/cp/owl/description.owl#')
rdfs=Namespace('http://www.w3.org/2000/01/rdf-schema#')
# Create the RDF graph
g = Graph()

# Bind the namespaces
g.bind("ear", ear)
g.bind("whata", whata)

# Define the document
doc = URIRef("https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl/doc1")
g.add((doc, RDF.type, ear.StringDocuverse))
g.add((doc, ear.hasContent, Literal("The West has no right to criticize our record on human rights, look at US actions in Central America, the history of slavery and of lynchings, not to mention apartheid in South Africa…", datatype=XSD.string)))

# Define the annotations
annotations = [
    (URIRef("https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl/Perspectivisation1"), "WhataboutistPerspectivisation", "The West has no right to criticize our record on human rights"),
     (URIRef("https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl/West"), "ResponsibilityOwner", "West"),
     (URIRef("https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl/Our"), "ResponsibilityOwner", "our"),
    (URIRef("https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl/HumanRightsViolation"), "BlamableEventuality", "record"),
    (URIRef("https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl/SlaveryAndLynchings"), "BlamableEventuality", "the history of slavery and of lynchings"),
    (URIRef("https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl/BlendedHumanRightsViolation"), "BlendedBlameFrame", "look at"),
     (URIRef("https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl/RespectingHumanRights"), "Ought", "human rights")
    ]

# Add the annotations to the graph
for ann, typ, text in annotations:
    g.add((ann, RDF.type, ear.Annotation))
    g.add((ann, ear.annotates, doc))
    g.add((ann, ear.hasXPathContext, Literal(f"//text()[contains(., '{text}')]", datatype=XSD.string)))
    g.add((ann, RDF.type, whata[typ]))

# Add triples for lens, eventuality, and their relationship
perspectivisation=URIRef('https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl/Perspectivisation1')
lens = URIRef("https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl/lens1")
eventuality1 = URIRef("https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl/HumanRightsViolation")
eventuality2 = URIRef("https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl/SlaveryAndLynchings")
ro1=URIRef("https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl/Our")
ro2=URIRef("https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl/West")
blended= URIRef("https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl/BlendedHumanRightsViolation")
ought= URIRef("https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl/RespectingHumanRights")

#add triples
g.add((lens, RDF.type, whata.WhataboutistLens))
g.add((lens, whata.shiftsFocusFrom, eventuality1))
g.add((lens, whata.relatedToLens, whata.ConfirmationBiasLens))
g.add((lens, whata.relatedToLens, whata.RelevanceFallacyLens))
g.add((lens, whata.shiftsFocusOn, eventuality2))
g.add((lens, whata.createsBlended, blended))
#g.add((eventuality1, RDF.type, whata.BlamableEventuality))
g.add((eventuality1, dns.satisfies, whata.BlameFrameDescription))
g.add((eventuality2, dns.satisfies, whata.BlameFrameDescription))
g.add((blended, dns.satisfies, whata.BlameFrameDescription))
g.add((blended, rdfs.subClassOf, whata.BlamableEventuality))
g.add((blended, rdfs.subClassOf, per.Blended))
g.add((ro1, whata.actsAgainst, eventuality1))
g.add((ro2, whata.actsAgainst, eventuality2))
g.add((perspectivisation, per.Eventuality, eventuality1))
g.add((perspectivisation, per.Lens, lens))
g.add((perspectivisation, whata.relatedIdentifyingPattern, Literal('no right to criticize')))
g.add((ought, whata.isParticipantIn, eventuality1))
g.add((ought, whata.isParticipantIn, eventuality2))
g.add((ro1, whata.isParticipantIn, eventuality1))
g.add((ro2, whata.isParticipantIn, eventuality2))





# Serialize the graph
#print(g.serialize(format="turtle"))
# Now you can perform the SPARQL query
q= """
PREFIX whata:<https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#> 
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX per:<http://www.ontologydesignpatterns.org/ont/persp/perspectivisation.owl#>
SELECT   ?actsUpon 
WHERE {
  
  ?s a whata:WhataboutistLens; 
  ?actsUpon ?eventuality.  
  FILTER (?actsUpon != rdf:type)
  FILTER (?actsUpon != rdfs:subClassOf)
}
      
"""

# Execute the query and print the results
for row in g.query(q):
    print(row)



(rdflib.term.URIRef('https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#shiftsFocusFrom'),)
(rdflib.term.URIRef('https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#relatedToLens'),)
(rdflib.term.URIRef('https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#relatedToLens'),)
(rdflib.term.URIRef('https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#shiftsFocusOn'),)
(rdflib.term.URIRef('https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#createsBlended'),)


In [70]:
#BLAMABLE EVENTUALITIES QUERY
# Now you can perform the SPARQL query
q2 = """
PREFIX whata:<https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#> 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX per:<http://www.ontologydesignpatterns.org/ont/persp/perspectivisation.owl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

SELECT ?s
WHERE {
  ?s a whata:BlamableEventuality.
 }
"""

# Execute the query and print the results
for row in g.query(q2):
    print(row)


(rdflib.term.URIRef('https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl/HumanRightsViolation'),)
(rdflib.term.URIRef('https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl/SlaveryAndLynchings'),)


In [132]:
#RESPONSIBILITY OWNERS QUERY
# Now you can perform the SPARQL query
q3 = """
PREFIX whata:<https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#> 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX per:<http://www.ontologydesignpatterns.org/ont/persp/perspectivisation.owl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

SELECT ?s
WHERE {
  ?s a whata:ResponsibilityOwner.
 }
"""

# Execute the query and print the results
for row in g.query(q3):
    print(row)


(rdflib.term.URIRef('https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl/West'),)
(rdflib.term.URIRef('https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl/Our'),)


In [136]:
#RESPONSIBILITY OWNERS QUERY
# Now you can perform the SPARQL query
q4 = """
PREFIX whata:<https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#> 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX per:<http://www.ontologydesignpatterns.org/ont/persp/perspectivisation.owl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

SELECT ?s ?o
WHERE {
  ?s whata:actsAgainst ?o.
 }
"""

# Execute the query and print the results
for row in g.query(q4):
    print(row)


(rdflib.term.URIRef('https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl/Our'), rdflib.term.URIRef('https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl/HumanRightsViolation'))
(rdflib.term.URIRef('https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl/West'), rdflib.term.URIRef('https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl/SlaveryAndLynchings'))


In [36]:
#RESPONSIBILITY OWNERS QUERY
# Now you can perform the SPARQL query
q5 = """
PREFIX whata:<https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#> 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX per:<http://www.ontologydesignpatterns.org/ont/persp/perspectivisation.owl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

SELECT ?s 
WHERE {
  ?s a whata:BlendedBlameFrame.
 }
"""

# Execute the query and print the results
for row in g.query(q5):
    print(row)


(rdflib.term.URIRef('https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl/BlendedHumanRightsViolation'),)


In [37]:
#RESPONSIBILITY OWNERS QUERY
# Now you can perform the SPARQL query
q5 = """
PREFIX whata:<https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#> 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX per:<http://www.ontologydesignpatterns.org/ont/persp/perspectivisation.owl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

SELECT ?p 
WHERE {
  ?s  whata:relatedIdentifyingPattern ?p.
 }
"""

# Execute the query and print the results
for row in g.query(q5):
    print(row)


(rdflib.term.Literal('no right to criticize'),)


# CQs

In [47]:
cq1 = """PREFIX whata:<https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#> 
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX per:<http://www.ontologydesignpatterns.org/ont/persp/perspectivisation.owl#>
    
SELECT  ?e
WHERE {
  
?s a whata:WhataboutistPerspectivisation.
?s per:Eventuality ?e.
}     
"""
results = g.query(cq1)

for row in results:
    print(row)
    

(rdflib.term.URIRef('https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl/HumanRightsViolation'),)


In [49]:

cq2 = """PREFIX whata:<https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#> 
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX per:<http://www.ontologydesignpatterns.org/ont/persp/perspectivisation.owl#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
SELECT ?s
WHERE {
  ?p a whata:WhataboutistPerspectivisation.
  ?p per:Eventuality ?e.
  ?s whata:shiftsFocusFrom ?e.
  
}
      
"""
results = g.query(cq2)

for row in results:
    print(row)
    

(rdflib.term.URIRef('https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl/lens1'),)


In [53]:
#what does whataboutism do? how it acts in the perspectivisation situation?

cq3 = """PREFIX whata:<https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#> 
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX per:<http://www.ontologydesignpatterns.org/ont/persp/perspectivisation.owl#>
SELECT   ?actsUpon
WHERE {
  ?s a whata:WhataboutistPerspectivisation.
  ?s per:Lens ?lens.
  ?s per:Eventuality ?eventuality.
  ?lens ?actsUpon ?eventuality.
}
      
"""
results = g.query(cq3)

for row in results:
    print(row)
    

(rdflib.term.URIRef('https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#shiftsFocusFrom'),)


In [55]:


cq3_a = """PREFIX whata:<https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#> 
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX per:<http://www.ontologydesignpatterns.org/ont/persp/perspectivisation.owl#>
SELECT   ?actsUpon 
WHERE {
  
  ?s a whata:WhataboutistLens; 
  ?actsUpon ?eventuality.  
  FILTER (?actsUpon != rdf:type)
  FILTER (?actsUpon != rdfs:subClassOf)
}
      
"""
results = g.query(cq3_a)

for row in results:
    print(row)
    

(rdflib.term.URIRef('https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#shiftsFocusFrom'),)
(rdflib.term.URIRef('https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#shiftsFocusOn'),)
(rdflib.term.URIRef('https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#createsBlended'),)


In [58]:
# Which linguistic elements do denote whataboutism?
#expected result: string with pattern, string with sentence
cq4 = """PREFIX whata:<https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#> 
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
SELECT  ?pattern 
WHERE {
  ?s a whata:WhataboutistPerspectivisation;
    
   whata:relatedIdentifyingPattern ?pattern.
   
}
      
"""
results = g.query(cq4)

for row in results:
    print(row)
    

(rdflib.term.Literal('no right to criticize'),)


In [67]:

#- Which kind of situation is the one involved in Whataboutism?

cq5 = """
    PREFIX whata:<https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#> 
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    SELECT  ?situation 
   WHERE {
       ?s whata:shiftsFocusFrom ?o.
       ?s whata:shiftsFocusOn ?o2.
       ?o a ?situation.
       ?o2 a ?situation.
        }
"""
results = g.query(cq5)

for row in results:
    print(row)
 

(rdflib.term.URIRef('http://www.essepuntato.it/2008/12/earmark#Annotation'),)
(rdflib.term.URIRef('https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#BlamableEventuality'),)


In [76]:
#- Which kind of situation is the one involved in Whataboutism?
# - What entities are involved in the blaming situation?
cq6 = '''
PREFIX whata: <https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX dns: <http://www.ontologydesignpatterns.org/cp/owl/description.owl#>

SELECT  ?description ?o
WHERE {
?s a whata:WhataboutistLens.
?s ?actsUpon ?o.
  ?o dns:satisfies ?description .
}

'''
results = g.query(cq6)

for row in results:
    print(row)
    

(rdflib.term.URIRef('https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#BlameFrameDescription'), rdflib.term.URIRef('https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl/HumanRightsViolation'))
(rdflib.term.URIRef('https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#BlameFrameDescription'), rdflib.term.URIRef('https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl/SlaveryAndLynchings'))
(rdflib.term.URIRef('https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#BlameFrameDescription'), rdflib.term.URIRef('https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl/BlendedHumanRightsViolation'))


In [84]:

# - What entities are involved in the blaming situation?
cq7 = """
    PREFIX whata:<https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#> 
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    SELECT distinct ?s
   WHERE {
       ?s whata:isParticipantIn ?blamable.
       ?blamable a whata:BlamableEventuality.
        }
"""
results = g.query(cq7)

for row in results:
    print(row)
    

(rdflib.term.URIRef('https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl/RespectingHumanRights'),)
(rdflib.term.URIRef('https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl/West'),)
(rdflib.term.URIRef('https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl/our'),)


In [85]:

#what does a blamable eventuality involve?
cq7_a = """PREFIX whata:<https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#> 
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX per:<http://www.ontologydesignpatterns.org/ont/persp/perspectivisation.owl#>
ASK
WHERE {
  ?s whata:isParticipantIn ?blamable.
  ?blamable a whata:BlamableEventuality.
  ?s a whata:Ought.
}
      
"""
results = g.query(cq7_a)

for row in results:
    print(row)
    

True


In [87]:

#what does a blamable eventuality involve?
cq7_b = """PREFIX whata:<https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#> 
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX per:<http://www.ontologydesignpatterns.org/ont/persp/perspectivisation.owl#>
ASK
WHERE {
  ?s whata:isParticipantIn ?blamable.
  ?blamable a whata:BlamableEventuality.
  ?s a whata:ResponsibilityOwner.
}
      
"""
results = g.query(cq7_b)

for row in results:
    print(row)
    

True


In [93]:

cq8 = """PREFIX whata:<https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#> 
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX per:<http://www.ontologydesignpatterns.org/ont/persp/perspectivisation.owl#>
ASK
WHERE {
  ?s a whata:WhataboutistLens.
  ?s whata:createsBlended ?o.
}
      
"""
results = g.query(cq8)

for row in results:
    print(row)
    

True


In [100]:

#what does a blamable eventuality involve?
cq9 = """PREFIX whata:<https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#> 
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX per:<http://www.ontologydesignpatterns.org/ont/persp/perspectivisation.owl#>
    PREFIX dns: <http://www.ontologydesignpatterns.org/cp/owl/description.owl#>
ASK
WHERE {
  ?s rdfs:subClassOf per:Blendable.
}
      
"""
results = g.query(cq9)

for row in results:
    print(row)
    

False


In [105]:
cq10 = """PREFIX whata:<https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#> 
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX per:<http://www.ontologydesignpatterns.org/ont/persp/perspectivisation.owl#>
     PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT  distinct  ?class
WHERE {
  ?s rdf:type whata:WhataboutistLens.
  ?s whata:createsBlended ?o.
  ?o rdf:type ?type.
  ?o rdfs:subClassOf ?class.

}
      
"""
results = g.query(cq10)

for row in results:
    print(row)
    

(rdflib.term.URIRef('https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#BlamableEventuality'),)
(rdflib.term.URIRef('http://www.ontologydesignpatterns.org/ont/persp/perspectivisation.owl#Blended'),)


In [119]:
cq11='''PREFIX whata: <https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX per:<http://www.ontologydesignpatterns.org/ont/persp/perspectivisation.owl#>
ask
WHERE{ }  
'''
results = g.query(cq11)

for row in results:
    print(row)

False


In [131]:
cq12_and_13='''PREFIX : <https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

select ?relatedLens
WHERE {
?l a whata:WhataboutistLens.
  ?l :relatedToLens ?relatedLens .
  
}'''
results = g.query(cq12_and_13)

for row in results:
    print(row)

(rdflib.term.URIRef('https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#ConfirmationBiasLens'),)
(rdflib.term.URIRef('https://raw.githubusercontent.com/Salvadana/whata_ontology/main/ontology/whataboutism_ontology.owl#RelevanceFallacyLens'),)
